# Análise do Perfil Nutricional Por Região

In [ ]:
%defaultDatasource jdbc:h2:mem:db

## Ingestão dos Dados do FooDB

<a href='https://foodb.ca/'>`https://foodb.ca/`</a>

O FooDB apresenta informações sobre a composição química e nutricional dos alimentos. Para a nossa análise, vamos considerar apenas alguns nutrientes.

In [ ]:
DROP VIEW IF EXISTS RecipeProfile;
DROP VIEW IF EXISTS RecipeIngredientFat;
DROP VIEW IF EXISTS RecipeIngredientProtein;
DROP VIEW IF EXISTS RecipeIngredientCarbo;
DROP VIEW IF EXISTS RecipeIngredients;
DROP VIEW IF EXISTS FoodNutrient;
DROP TABLE IF EXISTS Nutrient;
DROP TABLE IF EXISTS Food;
DROP TABLE IF EXISTS Nutrient_On_Food;

CREATE TABLE Nutrient (
    id INT,
    public_id VARCHAR(15),
    Nutrient_Name VARCHAR(50),
    PRIMARY KEY(id)
)AS SELECT id, public_id, LOWER(name) as Nutrient_Name
    FROM CSVREAD('../data/interim/ProcessedNutrient.csv');

CREATE TABLE Food (
    Id INT,
    Food_Name VARCHAR(50),
    Description VARCHAR(4000),
    Food_Group VARCHAR(50),
    Food_Subgroup VARCHAR(50),
    Public_Id VARCHAR(50),
    PRIMARY KEY(Id)
);

-- DROP INDEX IF EXISTS idx_tb_food_foodname;
-- CREATE INDEX idx_tb_food_foodname ON Food(Food_Name);

INSERT INTO Food (Id, Food_Name, Description, Food_Group, Food_Subgroup, Public_Id)
    SELECT Id, LOWER(name) as Food_Name, Description, Food_Group, Food_Subgroup, Public_Id
    FROM CSVREAD('../data/raw/foodb/Food.csv');
    
CREATE TABLE Nutrient_On_Food (
    Nutrient_Id INT,
    Food_Id INT,
    Orig_Content FLOAT,
    Unit VARCHAR(10),
    Quantity_Ref FLOAT,
    PRIMARY KEY(Nutrient_Id, Food_Id),
    FOREIGN KEY(Nutrient_Id)
        REFERENCES Nutrient(id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Food_Id)
        REFERENCES Food(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
);

-- DROP INDEX IF EXISTS idx_tb_nutrient_on_food_food_id;
-- CREATE INDEX idx_tb_nutrient_on_food_food_id ON Nutrient_On_Food(Food_Id);

INSERT INTO Nutrient_On_Food (Nutrient_Id, Food_Id, Orig_Content, Unit, Quantity_Ref)
    SELECT C.Source_Id AS Nutrient_Id, C.Food_Id, C.Orig_Content, C.Unit, C.Quantity_Ref
    FROM CSVREAD('../data/interim/ProcessedContent.csv') C, Food F, Nutrient N
    WHERE Food_Id=F.Id AND N.id = C.Source_Id;

## Ingestão dos Dados do CulinaryDB

<a href='https://cosylab.iiitd.edu.in/culinarydb/'>`https://cosylab.iiitd.edu.in/culinarydb`</a>
    

In [ ]:
DROP VIEW IF EXISTS RecipeProfile;
DROP VIEW IF EXISTS RecipeIngredientFat;
DROP VIEW IF EXISTS RecipeIngredientProtein;
DROP VIEW IF EXISTS RecipeIngredientCarbo;
DROP VIEW IF EXISTS RecipeIngredients;
DROP VIEW IF EXISTS FoodNutrient;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Ingredients;
DROP TABLE IF EXISTS Units;

CREATE TABLE Recipes (
    Recipe_ID INT,
    Title VARCHAR(250),
    Region VARCHAR(50),
    PRIMARY KEY(Recipe_ID)
) AS SELECT Recipe_ID, Title, Region
    FROM CSVREAD('../data/interim/Recipes.csv');

CREATE TABLE Units (
    Unit_Name VARCHAR(15),
    Grams FLOAT,
    PRIMARY KEY(Unit_Name)
) AS SELECT Unit_Name, Grams
    FROM CSVREAD('../data/interim/Units-Conversion.csv');

CREATE TABLE Ingredients (
    Recipe_ID INT,
    Food_ID INT,
    Food_Name VARCHAR(50),
    Quantity FLOAT,
    Unit VARCHAR(15),
    PRIMARY KEY(Recipe_ID, Food_ID),
    FOREIGN KEY(Recipe_ID)
        REFERENCES Recipes(Recipe_ID)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Food_ID)
        REFERENCES Food(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Unit)
        REFERENCES Units(Unit_name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
);

-- DROP INDEX IF EXISTS idx_tb_ingredients_food_name;
-- CREATE INDEX idx_tb_ingredients_food_name ON Ingredients(Food_Name);

INSERT INTO Ingredients (Recipe_Id, Food_Id, Food_Name, Quantity, Unit)
    SELECT Recipe_ID, Food_ID, LOWER(Aliased_Ingredient_Name) as Food_Name, Quantity, Unit
    FROM CSVREAD('../data/interim/IngredientOnFood2.csv');

# Pergunta e Análise 2:

- Quais os alimentos e receitas que mais contribuem para a ingestão de açúcares e gorduras e sódio para cada país/região?



In [ ]:
DROP TABLE IF EXISTS Recipe_Profile;

CREATE TABLE Recipe_Profile (
    Recipe_Id INT NOT NULL,
    weight FLOAT DEFAULT 0,
    fat FLOAT DEFAULT 0,
    carbo FLOAT DEFAULT 0,
    protein FLOAT DEFAULT 0,
    PRIMARY KEY(Recipe_Id),
    FOREIGN KEY(Recipe_Id)
        REFERENCES Recipes(Recipe_Id)
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
);

INSERT INTO Recipe_Profile (Recipe_Id)
    SELECT Recipe_Id FROM Recipes
        WHERE Recipe_Id IN (SELECT DISTINCT I.Recipe_Id FROM Ingredients I);

In [ ]:
DROP VIEW IF EXISTS RecipeIngredients;
CREATE VIEW RecipeIngredients AS
SELECT R.Recipe_ID, I.Food_Id, quantity * U.Grams as qnt_grams
    FROM Recipes R, Ingredients I, Units U
    WHERE R.Recipe_ID = I.Recipe_ID AND U.Unit_Name = I.Unit;

In [ ]:
DROP VIEW IF EXISTS FoodNutrient;
CREATE VIEW FoodNutrient AS
SELECT DISTINCT F.Food_Name, N.Nutrient_Name, NF.Food_Id, N.Id AS Nutrient_Id, Unit, Quantity_Ref, Orig_Content
    FROM Nutrient_On_Food NF, Nutrient N, Food F
    WHERE NF.Food_Id = F.id AND N.Id = NF.Nutrient_Id;

In [ ]:
DROP VIEW IF EXISTS RecipeIngredientFat;
DROP VIEW IF EXISTS RecipeIngredientProtein;
DROP VIEW IF EXISTS RecipeIngredientCarbo;


CREATE VIEW RecipeIngredientFat AS
SELECT RI.Recipe_Id, RI.Food_Id, FN.nutrient_id, RI.qnt_grams as ingredient_weight, RI.qnt_grams * (FN.orig_content/1000)/100 as fat
    FROM RecipeIngredients RI, FoodNutrient FN
    WHERE FN.nutrient_id = 1 AND RI.Food_Id = FN.Food_Id;
    
CREATE VIEW RecipeIngredientProtein AS
SELECT RI.Recipe_Id, RI.Food_Id, FN.nutrient_id, RI.qnt_grams as ingredient_weight, RI.qnt_grams * (FN.orig_content/1000)/100 as protein
    FROM RecipeIngredients RI, FoodNutrient FN
    WHERE FN.nutrient_id = 2 AND RI.Food_Id = FN.Food_Id;

CREATE VIEW RecipeIngredientCarbo AS
SELECT RI.Recipe_Id, RI.Food_Id, FN.nutrient_id, RI.qnt_grams as ingredient_weight, RI.qnt_grams * (FN.orig_content/1000)/100 as carbo
    FROM RecipeIngredients RI, FoodNutrient FN
    WHERE FN.nutrient_id = 3 AND RI.Food_Id = FN.Food_Id;

In [ ]:
UPDATE Recipe_Profile RP
    SET RP.fat = (SELECT COALESCE(SUM(RIF.fat), 0) FROM RecipeIngredientFat RIF WHERE RIF.Recipe_Id = RP.Recipe_Id),
        RP.protein = (SELECT COALESCE(SUM(RIP.protein), 0) FROM RecipeIngredientProtein RIP WHERE RIP.Recipe_Id = RP.Recipe_Id),
        RP.carbo = (SELECT COALESCE(SUM(RIC.carbo), 0) FROM RecipeIngredientCarbo RIC WHERE RIC.Recipe_Id = RP.Recipe_Id),
        RP.weight = (SELECT COALESCE(SUM(RI.qnt_grams), 0) FROM RecipeIngredients RI WHERE RI.Recipe_Id = RP.Recipe_Id);

In [ ]:
SELECT * FROM Recipe_Profile;